# Exercise 3

In [1]:
# MAIN LIBRARIES to use
using Optim
using Statistics
using ForwardDiff
using Plots
using LinearAlgebra
using CSV
using DataFrames
using StatsFuns

First we load the data that Nerlove collected in the early 60s:

In [2]:
data = DataFrame(CSV.File("nerlove.csv"))
data = log.(data[:,[:cost,:output,:labor,:fuel,:capital]]);

Now we assign y and x values:

In [3]:
n = size(data,1)
y = data[:,1]
x = data[:,2:end]
x[!,:intercept]=ones(size(data,1))
x = x[!,[:intercept,:output,:labor,:fuel,:capital]];   # add a column "intercept" of ones at index 1 

# Create X and y
y = convert(Array,y)                                   # turn both DataFrames into Arrays
x = convert(Array,x);

##  3 (a). Estimate the parameters using Restricted OLS using the CRTS restrictions

The general formulation of linear equality restrictions is the model:

$$
\begin{gather*}
    y=X\beta+\epsilon\\
    R\beta=r
\end{gather*}
$$

where $R$ is a $q\times k$ matrix ($q$ being the number of constraints on the parameters). In our case, since we want to explore the property of CRTS (constant returns to scale), given that we have a Cobb-Douglass function: 

$$
\begin{equation}
    \gamma = \frac{1}{\beta_q}\quad\Rightarrow\quad \boxed{\beta_q=1}
\end{equation}
$$

And thus, the matrix $R$ and $r$ will be defined as follows:

$$
\begin{equation}
    R=[0\,\,1\,\,0\,\,0\,\,0]\\
    r=1
\end{equation}
$$

### Restricted OLS funtamentals

$$
\begin{equation}
    \beta^{C}=\hat{\beta}+(X'X)^{-1}R'(R(X'X)^{-1}R')^{-1}(r-R\hat{\beta})
\end{equation}
$$

where $\hat{\beta}$ is the unconstrained OLS estimate - which we will have to compute in any case to get the constrained one. As far as the variance is concerned:

$$
\begin{equation}
    Var(\beta^{C})=M^{C}Var(\hat{\beta})(M^{C})'
\end{equation}
$$

where $M^{C}=I-(X'X)^{-1}R'(R(X'X)^{-1}R')^{-1}R$.

In [4]:
function ols(y::Array{Float64}, x::Array{Float64,2}; R=[], r=[], vc="white", silent=false)
        
    # compute ols coefficients, fitted values, and errors
    function lsfit(y, x)
        beta = inv(x'*x)*x'*y
        fit = x*beta
        errors = y - fit
        return beta, fit, errors
    end

    n,k = size(x)
    b, fit, e = lsfit(y,x)
    df = n-k
    sigsq = (e'*e/df)[1,1]
    xx_inv = inv(x'*x)
    ess = (e' * e)[1,1]
    
    # Restricted LS
    if R !=[]
        res_flag = true      # Restricted_flag True if restrictions
        q = size(R,1)
        P_inv = inv(R*xx_inv*R')
        b = b .- xx_inv*R'*P_inv*(R*b.-r)
        e = y-x*b;
        ess = (e' * e)[1,1]
        df = n-k-q
        sigsq = ess/df
        A = Matrix{Float64}(I, k, k) .- xx_inv*R'*P_inv*R;  # the matrix relating b and b_r
    end

    xe = x.*e
    varb = xx_inv*xe'xe*xx_inv

    # restricted LS
    if R !=[]
        varb = A*varb*A'
    end

    # We only need the SE
    seb = sqrt.(diag(varb))
    seb = seb.*(seb.>1e-16) # round off to zero when there are restrictions

    return b, seb, res_flag
end

ols (generic function with 1 method)

In [5]:
# Set restrictions: 
R = [0 1 0 0 0]        # CRTS if \beta_q=1
r = 1;

In [6]:
# ----------------- OLS ESTIMATION -----------------------

(b, seb, flg) = ols(y, x, R=R, r=r);

# Print results
if flg
    print("Restricted LS:\n\n")
    df = DataFrame(Estimators = ["beta_hat1", "beta_hat2", "beta_hat3", "beta_hat4", "beta_hat5"], Values = b, Standard_Errors=seb)
else
    print("Non-restricted LS:\n\n")
    println("Estimators: ",b)
    println("Standard Errors: ",seb)
end



Restricted LS:



,Estimators,Values,Standard_Errors
,String,Float64,Float64
1,beta_hat1,-7.53038,2.9195
2,beta_hat2,1.0,0.0
3,beta_hat3,0.0195454,0.375729
4,beta_hat4,0.715006,0.15923
5,beta_hat5,0.0758046,0.576294


## 3 (b). Calculate Wald, LR, and LM statistics and comment on the hypothesis test on restrictions.

In [7]:
function TestStatistics(y, x, R, r; silent=false)
    n,k = size(x)
    q = size(R,1)
    b = x\y
    xx_inv = inv(x'*x)
    P_inv = inv(R*xx_inv*R')
    b_r = b .- xx_inv*R'*P_inv*(R*b.-r)
    e = y - x*b
    ess = (e'*e)[1]
    e_r = y - x*b_r
    ess_r = (e_r' * e_r)[1]
    sigsqhat = ess/(n)
    sigsqhat_r = ess_r/(n)
    # Wald test (uses unrestricted model's est. of sig^2)
    W = (R*b.-r)'*P_inv*(R*b.-r)/sigsqhat
    # LR test
    lnl = -n/2*log(2*pi) - n/2*log(sigsqhat) - ess/(2.0*sigsqhat)
    lnl_r = -n/2*log(2*pi) - n/2*log(sigsqhat_r) - ess_r/(2.0*sigsqhat_r)
    LR = 2.0*(lnl-lnl_r)
    # Score test (uses restricted model's est. of sig^2)
    P_x = x * xx_inv * x'
    S = e_r' * P_x * e_r/(sigsqhat_r)
    
    tests_label = ["Wald","LR","LM"]
    tests = [W[1], LR[1], S[1]]
    pvalues = chisqccdf.(q,tests)
    
    return tests_label, tests, pvalues
end

TestStatistics (generic function with 1 method)

In [8]:
# -------------- TESTING --------------
t_label, tests, pval = TestStatistics(y, x, R, r)
df = DataFrame(Test_type = t_label, Test_values = tests, p_values = pval)

,Test_type,Test_values,p_values
,String,Float64,Float64
1,Wald,265.414,1.13343e-59
2,LR,150.863,1.12298e-34
3,LM,93.7713,3.54176e-22


### Commenting the results 

We obtain a very very small p-value: we have thus **strong evidence to reject the null hypothesis** at the usual significance level. In other words, the hypothesis that the technology exhibits CRTS must be rejected by analysing the data obtained from _nerlove.csv_. From this hypothesis, we can only state that the returns to scale in Electricity Suppy (from the data that Nerlove collected in the early 60s), are not of the type CRTS. 